# Home Automation Project

## Lights

The intensity of lights is dependent on the daylight.
The lights would get turned OFF gradually before sunrise, and they would get turned ON gradually before sunset.

In [10]:
from astral import LocationInfo
import pytz
from astral.geocoder import database, lookup

# Set up location information
tz_IND = pytz.timezone('Asia/Kolkata')
tz_LA = pytz.timezone('America/Los_Angeles')
# city = LocationInfo("Pune", "India", "Asia/Kolkata", 18.493255, 73.827113)
city = LocationInfo("Costa Mesa", "USA", "America/Los_Angeles", 33.692572, -117.878945)
# time_zone = lookup("Mumbai", database()).timezone
time_zone = lookup("Los_Angeles", database()).timezone
# tz = tz_IND
tz = tz_LA

# Define time_zone variable
time_zone = None

# Display location information
print((
    f"Information for {city.name}/{city.region}\n"
    f"Timezone: {city.timezone}\n"
    f"Latitude: {city.latitude:.02f}; Longitude: {city.longitude:.02f}\n"
))

Information for Costa Mesa/USA
Timezone: America/Los_Angeles
Latitude: 33.69; Longitude: -117.88



## Philips Hue

In [82]:
from apscheduler.schedulers.blocking import BlockingScheduler
import json
import requests
import urllib3
from astral.sun import sun
import time
from datetime import datetime, date, timedelta
from colour import Color

urllib3.disable_warnings()

hue_bridge_ip = '192.168.50.234'
user = 'kYezkr7YNGozEVSHkfneoHx3gO44ff8Ns0Qn9eKi'

s = sun(city.observer, tzinfo=time_zone)
sunrise = s['sunrise'].time()
sunset = s['sunset'].time()

data = None
hue_bridge_connection_error = False

def time_to_sunset_or_sunrise():
    global sunrise, sunset, data, hue_bridge_connection_error
    # Get current time
    now = datetime.now(tz)
    # print(f'Current time: {now.time()}')

    # Get time remaining for sunset and sunrise, and set intensity of light accordingly
    one_hour = timedelta(hours=1)
    one_minute = timedelta(minutes=1)
    zero_seconds = timedelta(seconds=0)

    # Check if time to sunset or sunrise is less than 1 hr, and set the brightness accordingly
    # Time to sunset
    time_to_sunset = datetime.combine(date.today(), sunset) - datetime.combine(date.today(), now.time())
    if time_to_sunset < one_hour and time_to_sunset > zero_seconds: # Less than 1 hr remaining for sunset
        # Calculate brightness of light in range 0-254
        brightness = ((one_hour - time_to_sunset)/one_minute)*254/60
        print(f'Brightness: {brightness}')
        time_to_sunset_less_than_1hr = True
    else:
        # Print time to sunset
        time_to_sunset_less_than_1hr = False
        if time_to_sunset < zero_seconds:
            time_to_sunset = datetime.combine((date.today()+timedelta(days=1)), sunset) - datetime.combine(date.today(), now.time())
        # print(f'Time to sunset: {time_to_sunset}')

    # Time to sunrise
    time_to_sunrise = datetime.combine(date.today(), sunrise) - datetime.combine(date.today(), now.time())
    if time_to_sunrise < one_hour and time_to_sunrise > zero_seconds: # Less than 1 hr remaining for sunrise
        # Calculate brightness of light in range 0-254
        brightness = (time_to_sunrise/one_minute)*254/60
        print(f'Brightness: {brightness}')
        time_to_sunrise_less_than_1hr = True
    else:
        # Print time to sunrise
        time_to_sunrise_less_than_1hr = False
        if time_to_sunrise < zero_seconds:
            time_to_sunrise = datetime.combine((date.today()+timedelta(days=1)), sunrise) - datetime.combine(date.today(), now.time())
        # print(f'Time to sunrise: {time_to_sunrise}\n')

    # Get the hue of the input color name
    # user_color = input("Enter color name: ")
    user_color = "white"
    color = Color(user_color)
    sat = color.saturation
    hue = color.hue
    hue = int(hue*65535) # Hue is in range 0-1, convert it to 0-65535
    sat = int(sat*254)  # Saturation is in range 0-1, convert it to 0-254
    if hue_bridge_connection_error == False:
        light_id = list(data.keys())        # Get the light ids
        # Turn on the lights
        for light in light_id:
            if(data[light]['state']['reachable'] == True and data[light]['state']['on'] == False):
                url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights/'+light+'/state'
                headers = {
                    'content-type': "application/json",
                    'cache-control': "no-cache"
                }
                sending_data ={'on': True}
                try:
                    response = requests.put(url, headers=headers, data=json.dumps(sending_data), verify=False)
                except requests.exceptions.Timeout:
                    hue_bridge_connection_error = True
                try:
                    url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights'
                    response = requests.get(url, verify=False)
                    data = json.loads(response.text)    # Convert json data to python dictionary
                except:
                    hue_bridge_connection_error = True
        # loop through all lights if time to sunrise or sunset is less than 1 hr and set the brightness accordingly
        if time_to_sunrise_less_than_1hr == True or time_to_sunset_less_than_1hr == True:
            for light in light_id:
                if(data[light]['state']['reachable'] == True):
                    url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights/'+light+'/state'
                    headers = {
                        'content-type': "application/json",
                        'cache-control': "no-cache"
                    }
                    sending_data ={'sat': sat, 'bri': brightness, 'hue': hue}
                    try:
                        response = requests.put(url, headers=headers, data=json.dumps(sending_data), verify=False)
                    except requests.exceptions.Timeout:
                        hue_bridge_connection_error = True
    else:
        try:
            url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights'
            response = requests.get(url, verify=False)
            data = json.loads(response.text)    # Convert json data to python dictionary
            light_id = list(data.keys())        # Get the light ids
            hue_bridge_connection_error = False
        except:
            hue_bridge_connection_error = True
            print("Hue Bridge Disconnected")

def sunrise_sunset_time():
    global sunrise, sunset, data, hue_bridge_connection_error
    url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights'
    try:
        response = requests.get(url, verify=False)
        data = json.loads(response.text)        # Convert json data to python dictionary
        # print(json.dumps(data, indent=4))     # Pretty print json data
    except:
        hue_bridge_connection_error = True
        print("Error in connecting to the Hue Bridge")

    # Get sunrise and sunset information
    s = sun(city.observer, tzinfo=time_zone)
    print((
        f'Sunrise: {s["sunrise"].time()}\n'
        f'Sunset:  {s["sunset"].time()}\n'
    ))
    sunrise = s['sunrise'].time()
    sunset = s['sunset'].time()

sunrise_sunset_time()
scheduler = BlockingScheduler()
scheduler.add_job(sunrise_sunset_time, 'interval', hours=24)        # Calculate sunrise and sunset times once a day
scheduler.add_job(time_to_sunset_or_sunrise, 'interval', minutes=1) # Calculate time to sunset or sunrise every minute and set the brightness of light accordingly
scheduler.start()

Sunrise: 06:10:00.779939
Sunset:  19:43:20.538813

Time to sunset: 21:27:37.549876
Time to sunrise: 7:54:17.791002

hue bridge error value =  False
Time to sunset: 21:27:32.549234
Time to sunrise: 7:54:12.790360

hue bridge error value =  False
Time to sunset: 21:27:27.549725
Time to sunrise: 7:54:07.790851

hue bridge error value =  False
Time to sunset: 21:27:22.549853
Time to sunrise: 7:54:02.790979

hue bridge error value =  False
Time to sunset: 21:27:17.549855
Time to sunrise: 7:53:57.790981

hue bridge error value =  False
Time to sunset: 21:27:12.549836
Time to sunrise: 7:53:52.790962

hue bridge error value =  False
Time to sunset: 21:27:07.549778
Time to sunrise: 7:53:47.790904

hue bridge error value =  False
Time to sunset: 21:27:02.549852
Time to sunrise: 7:53:42.790978

hue bridge error value =  False
Time to sunset: 21:26:57.549806
Time to sunrise: 7:53:37.790932

hue bridge error value =  False
Time to sunset: 21:26:52.549752
Time to sunrise: 7:53:32.790878

hue bridge 

KeyboardInterrupt: 

### Motion detected lights

There would be a LED light strip for each room which would light up subtlely and will be triggered by motion.

1. Routines:
    * If time is more than sunset and less than sunrise, then trigger lights that leads to the bathroom. These lights would be on the floor level.
    * A device (proxi sensor) to keep track of number of people in the house and in the room.
    

## Camera

This would keep track of number of people in the house.


## House AI

It would be updated with the number of people in the house, and in which room they are in. 

There would be microphones/with camera in each room listening to the commands And sending them to the central server.

The facial recognition will update number of persons in each room. (no facial recognition (for now)).

The voice would not have a recognition system (for now)

